![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [48]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Data Import and Sanity Check
df = pd.read_csv('rental_info.csv')
# df.info()
date_columns = ['rental_date', 'return_date']
int_columns = ['release_year']
for column in df[date_columns]:
    df[column] = df[column].astype('datetime64')
df['release_year'] = df['release_year'].astype('int')

# Calculating rental length
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

# Creating dummy variables for "special features" column
df['special_features'] = df['special_features'].str.strip('{}').str.split(',')
df['special_features'] = df['special_features'].apply(lambda x: [item.strip('""') for item in x])
df['deleted_scenes'] = df['special_features'].apply(lambda x: 1 if 'Deleted Scenes' in x else 0)
df['behind_the_scenes'] = df['special_features'].apply(lambda x: 1 if 'Behind the Scenes' in x else 0)

# Separating features from target variables and training from test sets
X = df.drop(['rental_length_days', 'special_features', 'rental_date', 'return_date'], axis=1)
y = df['rental_length_days']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Training the model
linreg = LinearRegression()
best_model = linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
best_mse = mean_squared_error(y_test, y_pred)

print(best_mse)

2.9417238646975883
